In [ ]:
import requests
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092')

response = requests.get('https://rest.coinapi.io/v1/exchangerate/BTC/USD')
producer.send('my-data-stream', value=response.content)
producer.flush()

In [2]:
from confluent_kafka import Producer

producer = Producer({'bootstrap.servers': 'localhost:9092'})

response = requests.get('https://rest.coinapi.io/v1/exchangerate/BTC/USD')
producer.produce('my-data-stream', value=response.content)
producer.flush()


0

In [4]:
import requests
from confluent_kafka import Producer

# Replace 'YOUR_API_KEY' with your actual CoinAPI key
headers = {'X-CoinAPI-Key': '8e64a7e8-441f-44ee-95a7-a80d00d494f1'}

# Make the API request with the headers
response = requests.get('https://rest.coinapi.io/v1/exchangerate/BTC/USD', headers=headers)

# Check for a valid response before sending to Kafka
if response.status_code == 200:
    producer = Producer({'bootstrap.servers': 'localhost:9092'})
    producer.produce('my-data-stream', value=response.content)
    #producer.produce('crypto-exchange-rates', value=response.content)
    producer.flush()
else:
    print(f"Error: {response.status_code}, {response.text}")


%3|1727427136.170|FAIL|rdkafka#producer-4| [thrd:jf-hp:9092/0]: jf-hp:9092/0: Failed to connect to broker at [jf-hp]:9092: Invalid argument (after 0ms in state CONNECT)
%3|1727427137.172|FAIL|rdkafka#producer-4| [thrd:jf-hp:9092/0]: jf-hp:9092/0: Failed to connect to broker at [jf-hp]:9092: Invalid argument (after 1ms in state CONNECT, 1 identical error(s) suppressed)


%5|1727429296.232|REQTMOUT|rdkafka#producer-4| [thrd:jf-hp:9092/0]: jf-hp:9092/0: Timed out MetadataRequest in flight (after 60054ms, timeout #0)
%4|1727429296.232|REQTMOUT|rdkafka#producer-4| [thrd:jf-hp:9092/0]: jf-hp:9092/0: Timed out 1 in-flight, 0 retry-queued, 0 out-queue, 0 partially-sent requests
%3|1727429296.232|FAIL|rdkafka#producer-4| [thrd:jf-hp:9092/0]: jf-hp:9092/0: 1 request(s) timed out: disconnect (average rtt 1.223ms) (after 2158846ms in state UP)


In [1]:
import requests
from confluent_kafka import Producer
import json

# Replace 'YOUR_API_KEY' with your actual CoinAPI key
headers = {'X-CoinAPI-Key': '8e64a7e8-441f-44ee-95a7-a80d00d494f1'}

# Kafka producer setup
producer = Producer({'bootstrap.servers': 'localhost:9092'})

def fetch_ohlcv_data():
    # Endpoint for latest OHLCV data for a specific symbol (e.g., BTC/USD)
    response = requests.get('https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_BTC_USD/latest', headers=headers)
    
    # Check for a valid response
    if response.status_code == 200:
        data = response.json()
        producer.produce('ohlcv-data', value=json.dumps(data))
        producer.flush()
    else:
        print(f"Error: {response.status_code}, {response.text}")

if __name__ == '__main__':
    fetch_ohlcv_data()


Error: 429, {
  "title": "Forbidden",
  "status": 429,
  "detail": "Quota exceeded: REST API Requests per day.",
  "error": "Forbidden (Quota exceeded: REST API Requests per day.)",
  "QuotaKey": "CAPI_DTA_REST_REQ_QUOTA_DAILY",
  "QuotaName": "REST API Requests per day",
  "QuotaType": "Quota",
  "QuotaValueCurrentUsage": 155,
  "QuotaValue": 100,
  "QuotaValueUnit": "Credits",
  "QuotaValueAdjustable": "Yes, upgrade subscription or enable pay-as-you-go billing above the organizaiton quota."
}
